In [1]:
import sys
import pysqlite3

sys.modules['sqlite3'] = pysqlite3

import os
import time
import warnings
import ollama
from functools import cached_property
from langchain_community.llms import AzureOpenAI
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain_text_splitters import (Language,RecursiveCharacterTextSplitter)
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
import google.generativeai as genai
from groq import Groq
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import requests
import voyageai
from langchain.retrievers import ContextualCompressionRetriever
from langchain_voyageai import VoyageAIEmbeddings,VoyageAIRerank
from tree_sitter_languages import get_language, get_parser
from llama_index.core.text_splitter import CodeSplitter
from dotenv import load_dotenv
import subprocess

from chunker import get_code_chunks

from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


True

In [2]:
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata

def doc_merger(splits):
    current = 0
    while True:
        doc_lines = len(splits[current].splitlines())
        if doc_lines < 3:
            # merge with next doc
            splits[current] += splits[current + 1]
            splits.pop(current + 1)
        else:
            current += 1
        
        if current == len(splits) - 1:
            return splits

command = ["clang-format","-style={ColumnLimit: 300, AllowShortFunctionsOnASingleLine: All, AllowShortIfStatementsOnASingleLine: true}","-i","original.txt"]

subprocess.run(command, check=True)

file_path = "original.txt"
with open(file_path, "r") as f:
    docs = f.read()

splits = get_code_chunks(docs)
new_splits = [split for split in splits if len(split) > 2]
new_splits2 = doc_merger(new_splits)
# documents = [Document(page_content=split) for split in splits]
documents = [Document(page_content=split) for split in new_splits2]
# save documents to files
# for i, doc in enumerate(documents):
#     with open(f"docs/doc_{i}.txt", "w") as f:
#         f.write(doc.page_content)
os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
db = Chroma.from_documents(documents=documents, embedding=embeddings)


In [3]:
def combine_docs(docs):
    return "\n\n".join(f"Snippet.{i+1}:\n\n{doc.page_content}" for i, doc in enumerate(docs))


In [4]:
safe = [
{
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE",
},
{
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE",
},
{
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE",
},
{
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE",
},
]
genai.configure(api_key=os.environ.get("GENAI_API_KEY"))
generation_config = {
"temperature": 0.1,
"top_p": 0.95,
"top_k": 64,
"max_output_tokens": 8192,
"response_mime_type": "text/plain",
}
model = genai.GenerativeModel(
model_name="gemini-1.5-pro",
generation_config=generation_config,
safety_settings = safe
)
llm = model.start_chat(history=[])

In [6]:
with open("../generality_cases/mkdir/mkdir-5.2.1.c.origin.c", "r") as f:
    code = f.read()
prompt = code + "\n\n" + """Above is the complete mkdir coreutil code. I will give you snippets, your job is to explain to be what their role is. I am mainly concerned with the functionality of the -m flag. Say yes when ready"""
llm.send_message(prompt).text

'Yes, I am ready. Please provide the code snippets you want me to explain. I will do my best to clarify their role in the `mkdir` coreutil, especially in relation to the `-m` flag. \n'

In [18]:
prompt = """"int same_name(char const *source, char const *dest) {
  char const *source_basename;
  char *tmp;
  char const *dest_basename;
  char *tmp___0;
  size_t source_baselen;
  size_t tmp___1;
  size_t dest_baselen;
  size_t tmp___2;
  _Bool identical_basenames;
  int tmp___3;
  int tmp___4;
  _Bool compare_dirs;
  _Bool same;
  struct stat source_dir_stats;
  struct stat dest_dir_stats;
  char *source_dirname;
  char *dest_dirname;
  int *tmp___5;
  int tmp___6;
  int *tmp___7;
  int tmp___8;
  int tmp___9;

  {
    {
      tmp = base_name(source);
      source_basename = (char const *)tmp;
      tmp___0 = base_name(dest);
      dest_basename = (char const *)tmp___0;
      tmp___1 = base_len(source_basename);
      source_baselen = tmp___1;
      tmp___2 = base_len(dest_basename);
      dest_baselen = tmp___2;
    }
    if (source_baselen == dest_baselen) {
      {
        tmp___3 = memcmp((void const *)source_basename, (void const *)dest_basename, dest_baselen);
      }
      if (tmp___3 == 0) {
        tmp___4 = 1;
      } else {
        tmp___4 = 0;
      }
    } else {
      tmp___4 = 0;
    }
    identical_basenames = (_Bool)tmp___4;
    compare_dirs = identical_basenames;
    same = (_Bool)0;
    if (compare_dirs) {
      {
        source_dirname = dir_name(source);
        dest_dirname = dir_name(dest);
        tmp___6 = stat((char const * /* __restrict  */)((char const *)source_dirname), (struct stat * /* __restrict  */)(&source_dir_stats));
      }
      if (tmp___6) {
        {
          tmp___5 = __errno_location();
          error(1, *tmp___5, "%s", source_dirname);
        }
      }
      { tmp___8 = stat((char const * /* __restrict  */)((char const *)dest_dirname), (struct stat * /* __restrict  */)(&dest_dir_stats)); }
      if (tmp___8) {
        {
          tmp___7 = __errno_location();
          error(1, *tmp___7, "%s", dest_dirname);
        }
      }
      if (source_dir_stats.st_ino == dest_dir_stats.st_ino) {
        if (source_dir_stats.st_dev == dest_dir_stats.st_dev) {
          tmp___9 = 1;
        } else {
          tmp___9 = 0;
        }
      } else {
        tmp___9 = 0;
      }
      {
        same = (_Bool)tmp___9;
        rpl_free((void *)source_dirname);
        rpl_free((void *)dest_dirname);
      }
    }
    return ((int)same);
  }
}
what does the above function do in reference to the -m flag?"""


print(llm.send_message(prompt).text)

The `same_name` function is not directly related to the `-m` flag in `mkdir`. It's a utility function that determines if two file paths refer to the same file on the filesystem, even if the paths themselves are different. 

Here's a breakdown of what the function does:

1. **Extracts Basenames:** It extracts the base filenames (the last component of the path) from both `source` and `dest` paths.
2. **Compares Basenames:** It compares the basenames. If they are different, the function immediately returns 0 (false), indicating the paths do not refer to the same file.
3. **Extracts Directories:** If the basenames are identical, it extracts the directory portions of both paths.
4. **Retrieves File Stats:** It uses the `stat` function to retrieve file status information (including inode number and device ID) for both directories.
5. **Compares Inode and Device:** It compares the inode numbers and device IDs of the two directories. If both are identical, it means the directories are the same

In [10]:
import gc

from sklearn.metrics.pairwise import cosine_similarity

os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY")


def call_retrieval_sada(pretext, fifty_clean):

    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

    # Use similarity_search_with_score instead of invoke
    retrieved_docs_with_scores = db.similarity_search_with_score(pretext, k=4)
    
    retrieved_docs = [doc for doc, _ in retrieved_docs_with_scores]
    similarity_scores = [score for _, score in retrieved_docs_with_scores]


    # Sort both lists based on similarity scores in descending order
    sorted_pairs = sorted(zip(retrieved_docs, similarity_scores), key=lambda x: x[1], reverse=True)

    # Unzip the sorted pairs
    retrieved_docs, similarity_scores = zip(*sorted_pairs)

    # Convert back to lists if needed
    retrieved_docs = list(retrieved_docs)
    similarity_scores = list(similarity_scores)
    
    formatted_context = combine_docs(retrieved_docs)
    formatted_context_2 = "Snippet.0: \n\n" + fifty_clean + "\n\n" + formatted_context
    
    return formatted_context_2, similarity_scores, retrieved_docs


    # retrieved_docs = retriever.invoke(pretext)

    # formatted_context = combine_docs(retrieved_docs)

    # formatted_context_2 = "Snippet.0: \n\n" + fifty_clean + "\n\n" + formatted_context

    # return formatted_context_2

    # return formatted_context, [], retrieved_docs, []




### Test your cand sets below

##### Add `Pretext`, `Fifty_Text` and `Query` as variables

##### Original.txt, prompt, and sec_list must be in `manual_testing/` directory

In [17]:
llm = model.start_chat(history=[])

with open('functionality_prompt.txt', 'r') as file:
    prompt_template = file.read()

# with open('sec_list.txt', 'r') as file:
#     sec_list = file.read()


pretext = """error_msg = E_invalid_user;
                    }
                }
            }
            else {
                unum = pwd->pw_uid;
                if ((unsigned long)g == (unsigned long)((void *)0)) {
                    if ((unsigned long)separator != (unsigned long)((void *)0)) {
                        gnum = pwd->pw_gid;
                        grp = getgrgid(gnum);
                        if (grp) {
                            tmp___5 = grp->gr_name;
                        }
                        else {
                            tmp___4 = umaxtostr((uintmax_t)gnum, buf___1);
                            tmp___5 = tmp___4;
                        }
                        gname = xstrdup((char const *)tmp___5);
                        endgrent();
                    }
                }
            }
            endpwent();
        }
        if ((unsigned long)g != (unsigned long)((void *)0)) {
            if ((unsigned long)error_msg == (unsigned long)((void *)0)) {"""

fifty_clean = """                        error_msg = E_invalid_user;
                    }
                }
            }
            else {
                unum = pwd->pw_uid;
                if ((unsigned long)g == (unsigned long)((void *)0)) {
                    if ((unsigned long)separator != (unsigned long)((void *)0)) {
                        gnum = pwd->pw_gid;
                        grp = getgrgid(gnum);
                        if (grp) {
                            tmp___5 = grp->gr_name;
                        }
                        else {
                            tmp___4 = umaxtostr((uintmax_t)gnum, buf___1);
                            tmp___5 = tmp___4;
                        }
                        gname = xstrdup((char const *)tmp___5);
                        endgrent();
                    }
                }
            }
            endpwent();
        }
        if ((unsigned long)g != (unsigned long)((void *)0)) {
            if ((unsigned long)error_msg == (unsigned long)((void *)0)) {
                if ((int const) * g == 43) {
                    grp = (struct group *)((void *)0);
                }
                else {
                    tmp___6 = getgrnam(g);
                    grp = tmp___6;
                }
                if ((unsigned long)grp == (unsigned long)((void *)0)) {
                    tmp___8 = xstrtoul(g, (char **)((void *)0), 10, &tmp___7, "");
                    if ((unsigned int)tmp___8 == 0U) {
                        if (tmp___7 <= 4294967295UL) {
                            if ((gid_t)tmp___7 != 4294967295U) {
                                gnum = (gid_t)tmp___7;
                            }
                            else {
                                error_msg = E_invalid_group;
                            }
                        }
                        else {
                            error_msg = E_invalid_group;
                        }
                    }
                    else {
                        error_msg = E_invalid_group;
                    }
                }
                else {
                    gnum = grp->gr_gid;
                }
                endgrent();
                gname = xstrdup(g);
            }
        }
        if ((unsigned long)error_msg == (unsigned long)((void *)0)) {
            *uid = unum;
            *gid = gnum;
            *username = u;
            *groupname = gname;
            u = (char *)((void *)0);
        }
        else {
            free((void *)gname);
        }
        free((void *)u);
        tmp___9 = (char const *)gettext(error_msg);
        return (tmp___9);
    }
}
char const *parse_user_spec(char const *spec, uid_t *uid, gid_t *gid,
                            char **username, char **groupname)
{
    char const *colon;
    char const *tmp;
    char const *error_msg;
    char const *tmp___0;
    char const *dot;
    char const *tmp___1;
    char const *tmp___2;
    {"""

query = """if (grp) {
  tmp___5 = grp->gr_name;
} else {
  tmp___4 = umaxtostr((uintmax_t)gnum, buf___1);
  tmp___5 = tmp___4;
}
"""
formatted_context, scores, individual_docs = call_retrieval_sada(pretext, fifty_clean)
with open('req_list.txt', 'r') as file:
    req_list = file.read()
prompt = prompt_template.format(req_list=req_list, context=formatted_context, query=query)

response = llm.send_message(prompt).text

print("\nRESPONSE:\n",response)



RESPONSE:
 Class 1: N/A - This code snippet is related to user/group handling.
Class 2: N/A - This code snippet is directly involved in resolving group names.
Class 3: This code snippet contributes to resolving group names, which is necessary for changing group ownership.
Class 4: N/A - While important for resolving group names, its absence might not completely break the core functionality.

Explanation: This code snippet is responsible for retrieving the group name either from the 'grp' structure if available or converting the numeric group ID (gnum) to a string representation using 'umaxtostr'. This is part of the functionality for handling symbolic or numeric group specifications.

Final Verdict: 3 



In [13]:
# response = llm.send_message("Wpuldnt this code be important when i am changing the User: or :GROUP recursively").text

# print("\nRESPONSE:\n",response)

print(prompt)

You are a proficient C code evaluator tasked with assessing the importance of a specific code snippet within the larger context of the overall program. 

Evaluation Scale:

- Class 1: Completely Unnecessary Code - The code is part of unrequired or unrelated functionality and does not contribute in any way to the program's primary purpose or desired behavior. Removing this code will not impact the required functionality.
- Class 2: Somewhat Unnecessary Code - This code might be involved in general operations but does not directly contribute to or affect the required functionality.  It may help with performance, readability, or minor features but can be removed without breaking the core required functionality.
- Class 3: Necessary Code - This code snippet directly contributes to the core required functionality or main purpose of the program. It plays an active role in achieving specific tasks that are considered necessary for the correct operation of the program.
- Class 4: Critical Code

In [ ]:
print(scores)
print(len(set([doc.page_content for doc in individual_docs])))

print(formatted_context)

[0.4558253884315491, 0.43654656410217285, 0.4288574457168579, 0.3548514246940613]
4
Snippet.0: 

        is_dotdot = (_Bool)tmp___0;
        if (sp->fts_options & 4) {
            if (sp->fts_options & 512) {
                if (0 <= fd) {
                    close(fd);
                }
            }
            return (0);
        }
        if (fd < 0) {
            if (is_dotdot) {
                if (sp->fts_options & 512) {
                    tmp___1 = i_ring_empty((I_ring const *)(&sp->fts_fd_ring));
                    if (!tmp___1) {
                        parent_fd = i_ring_pop(&sp->fts_fd_ring);
                        is_dotdot = (_Bool)1;
                        if (0 <= parent_fd) {
                            fd = parent_fd;
                            dir = (char const *)((void *)0);
                        }
                    }
                }
            }
        }
        newfd = fd;
        if (fd < 0) {
            newfd = diropen((FTS const *)sp, dir);
     

In [ ]:
llm = model.start_chat(history=[])

with open('prompt_in_coverage.txt', 'r') as file:
    prompt_template = file.read()

with open('sec_list.txt', 'r') as file:
    sec_list = file.read()


pretext = """            }
            if ((int)c == 51) {
                goto case_37;
            }
            if ((int)c == 52) {
                goto case_37;
            }
            if ((int)c == 53) {
                goto case_37;
            }
            if ((int)c == 54) {
                goto case_37;
            }
            if ((int)c == 55) {
                goto case_37;
            }
            if ((int)c == 56) {
                goto case_37;
            }
            if ((int)c == 57) {
                goto case_37;
            }"""

fifty_clean = """            }
            if ((int)c == 51) {
                goto case_37;
            }
            if ((int)c == 52) {
                goto case_37;
            }
            if ((int)c == 53) {
                goto case_37;
            }
            if ((int)c == 54) {
                goto case_37;
            }
            if ((int)c == 55) {
                goto case_37;
            }
            if ((int)c == 56) {
                goto case_37;
            }
            if ((int)c == 57) {
                goto case_37;
            }
            if ((int)c == 58) {
                goto case_37;
            }
            if ((int)c == 65) {
                goto case_37;
            }
            if ((int)c == 66) {
                goto case_37;
            }
            if ((int)c == 67) {
                goto case_37;
            }
            if ((int)c == 68) {
                goto case_37;
            }
            if ((int)c == 69) {
                goto case_37;
            }
            if ((int)c == 70) {
                goto case_37;
            }
            if ((int)c == 71) {
                goto case_37;
            }
            if ((int)c == 72) {
                goto case_37;
            }
            if ((int)c == 73) {
                goto case_37;
            }
            if ((int)c == 74) {
                goto case_37;
            }
            if ((int)c == 75) {
                goto case_37;
            }
            if ((int)c == 76) {
                goto case_37;
            }
            if ((int)c == 77) {
                goto case_37;
            }
            if ((int)c == 78) {
                goto case_37;
            }
            if ((int)c == 79) {
                goto case_37;
            }
            if ((int)c == 80) {
                goto case_37;
            }
            if ((int)c == 81) {
                goto case_37;
            }
            if ((int)c == 82) {
                goto case_37;
            }
            if ((int)c == 83) {
                goto case_37;
            }"""

query = """if ((int)c == 54) {
  goto case_37;
}"""
formatted_context, scores, individual_docs = call_retrieval_sada(pretext, fifty_clean)

prompt = prompt_template.format(sec_list=sec_list, formatted_context=formatted_context, query=query)

response = llm.send_message(prompt).text

print("\nRESPONSE:\n",response)



RESPONSE:
 Class 1: This code snippet does not seem to be related to any unrequired functionalities.
Class 2: This code snippet does not seem to be somewhat unnecessary.
Class 3: This code snippet is part of a switch-case structure handling different characters, potentially affecting how file paths are processed during recursive ownership/group change.
Class 4: This code snippet is not critical, as its absence might only lead to incorrect handling of specific characters and not a complete program break.

Explanation: The code snippet is part of a larger function (as evidenced by the `goto` statements and variable usage) that seems to be processing characters, possibly in the context of file paths. This is relevant to the recursive ownership/group change functionality, as it involves handling file paths. However, the specific character handling might not be critical, and its absence might only lead to incorrect processing of certain characters in file paths.

Final Verdict: Class 3 



In [ ]:
print(scores)
print(len(set([doc.page_content for doc in individual_docs])))

# print(formatted_context)
for doc in individual_docs:
    print(doc.page_content)
    print("\n\n")

[0.6145888566970825, 0.49018681049346924, 0.4783892035484314, 0.47674670815467834]
4
      if ((int)c == 116) {
        goto case_37;
      }
      if ((int)c == 117) {
        goto case_37;
      }
      if ((int)c == 118) {
        goto case_37;
      }
      if ((int)c == 119) {
        goto case_37;
      }
      if ((int)c == 120) {
        goto case_37;
      }
      if ((int)c == 121) {
        goto case_37;
      }
      if ((int)c == 122) {
        goto case_37;
      }
      goto switch_default___2;
    case_0___0:
      if (backslash_escapes) {
        if (elide_outer_quotes) {
          goto force_outer_quoting_style;
        }
        while (1) {

          if (len < buffersize) {
            *(buffer + len) = (char)'\\';
          }
          len++;
          goto while_break___4;
        }
      while_break___4:;
        if (i + 1UL < argsize) {
          if (48 <= (int)*(arg + (i + 1UL))) {
            if ((int const) * (arg + (i + 1UL)) <= 57) {
              while (1)